In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../../..")

In [3]:
import os
from datetime import datetime, UTC
import analytiq_data as ad
import asyncio
from bson import ObjectId
from dotenv import load_dotenv
import docrouter_app.payments

In [4]:
ad.common.setup()
await docrouter_app.payments.init_payments_env()

In [ ]:
#await docrouter_app.payments.delete_all_payments_customers(dryrun=False)

In [26]:
org_id = "6795345439604beca2b2808d"
org_id = "67b00800e1dd3da6cb64f77d"
org_id = "679533ee39604beca2b2803a"
org_id = "679eaf804cfaaaa3640811f1"

In [ ]:
await docrouter_app.payments.sync_payments_customer(org_id=org_id)

In [22]:
stripe_customer = await docrouter_app.payments.get_payments_customer(org_id=org_id)

In [ ]:
stripe_customer

In [ ]:
stripe_customer.id

In [ ]:
subscription = await docrouter_app.payments.get_subscription(stripe_customer.id)
if subscription:
    print(docrouter_app.payments.get_subscription_type(subscription))
else:
    print("No subscription found")

In [ ]:
await docrouter_app.payments.get_subscription(customer_id=stripe_customer.id)

In [ ]:
await docrouter_app.payments.set_subscription_type(stripe_customer.id, "individual")

In [ ]:
price_id = docrouter_app.payments.TIER_TO_PRICE["individual"]
price_id

In [16]:
# Use the new subroutine
subscription_type = docrouter_app.payments.get_subscription_type(price_id)

# Create the subscription
subscription = await docrouter_app.payments.StripeAsync.subscription_create(
    customer=stripe_customer.id,
    items=[{
        "price": price_id,
    }],
    payment_behavior="default_incomplete",
    expand=["latest_invoice.payment_intent"],
)

# Find the subscription item ID for usage reporting later
subscription_item_id = subscription["items"]["data"][0]["id"]

In [ ]:
subscription

In [ ]:
await docrouter_app.payments.record_usage(org_id=org_id, pages_processed=10, operation="test")


